# Investments Project (Spring 2024)

**Authors:**
- Marc-Antoine Allard
- Adam Zinebi
- Paul Teiletche
- ...

**DUE Date: June 21 at 23:59**

---
# Utils

## Imports

## Helper Functions

---
# The Data

**Download:**
- Monthly stock returns from CRSP from January 1, 1964, to December 31, 2023, for all common stocks traded on the NYSE and AMEX.
- The value-weighted CRSP market return and 1-month T-bill returns as a risk-free rate.

---
# Betting against Beta strategy (BaB)

---
# Momentum Strategy (Mom)

---
# Idiosyncratic Volatility Strategy (IV)

---
# Optimal Fund Portfolio Return (STRAT)

---
# Performance and risk analysis for the Fund strategy

---
# Industry neutral strategy